# TIME SERIES REGRESSION FAVORITA

In this project I will be assuming a role of a data scientist in Corporation Favorita, a large Ecuadorian-based grocery retailer. Corporation Favorita wants to ensure that they always have the right quantity of products in stock. To do this i have decided to build a series of machine learning models to forecast the demand of products in various locations. The marketing and sales team have provided some data to aid this endeavor. I will be using CRISP-DM Framework for this project.

## Business Understanding

It has always been a problem for companies to determine the right level of stock to have. There are a lot of factors to consider when it comes to stocking: lead times, cost of transportation, cost of warehousing, product lifespan and others.

Company's want to know the best level of stock to have in order to be able to satisfy customers demand whilst spend the lease possible amount on the stock.

Stock is company resources (money) held up, it is therefore prudent to be able to determine the right level of stock to hold. Policies like JIT(Just in time) and EOQ(Economic order quantities) have been utilised to manage stock levels.

Overstocking can lead to: locked up funds that could be used for other projects, expiry of goods, large/special warehouse spaces (which is costly), retailers like Favorita sometimes have to reduce prices of near-expiry products to cut down on losses. Some dangers of under-stocking could include disatisfied customers (which may lead to them voting with their feet), decrease in revenue, companies may also end up spending more on logistics when they understock frequently.

In this project we will be using machine learning and regression models to forecast sales which would help us determine stock levels to hold, the company believes this would be a more accurate policy since it is based on a large dataset of previous sales.

## Research questions

1. Is the train dataset complete (has all the required dates)?
2. Which dates have the lowest and highest sales for each year (excluding days the store was closed)?
3. Did the earthquake impact sales?
4. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
5. Are sales affected by promotions, oil prices and holidays?
6. What analysis can we get from the date and its extractable features?
7. Which product family and stores did the promotions affect.
8. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
9. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.
10. Compare the sales for each month across the years and determine which month of which year had the highest sales.

## Hypothesis

Null Hypothesis (H0): "There is no significant relationship between store sales and products promotions."

Alternative Hypothesis (Ha): "There is a significant relationship between store sales and products promotions."

In [13]:
#Data extraction and manipulation

import pyodbc    
from dotenv import dotenv_values
import pandas as pd
from itertools import product
import numpy as np


#Visualization and modelling
import matplotlib.dates as mdates
import calplot
from statsmodels.tsa.seasonal import seasonal_decompose
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
#Utilities
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
#from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
#import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, acf, pacf
import itertools
import warnings 
warnings.filterwarnings('ignore')
import os



In [14]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER_NAME")
database = environment_variables.get("DATABASE_NAME")
username = environment_variables.get("USER")
password = environment_variables.get("PASSWORD")

In [15]:
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"


In [17]:

# Use the connect method of the pyodbc library and pass in the connection string.
connection = pyodbc.connect(connection_string)

In [18]:
# Sql query to retrieve data from Microsoft SQL database
oil_df = "SELECT * FROM dbo.oil"
#Execute the SQL queries and load the results into Pandas DataFrames
oil_df = pd.read_sql(oil_df, connection)
#Show retrieved table
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [19]:
# Sql query to retrieve data from Microsoft SQL database
holiday_events_df = "SELECT * FROM dbo.holidays_events"
#Execute the SQL queries and load the results into Pandas DataFrames
holiday_events_df = pd.read_sql(holiday_events_df, connection)
#Show retrieved table
holiday_events_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [20]:
# Sql query to retrieve data from Microsoft SQL database
store = "SELECT * FROM dbo.stores"
#Execute the SQL queries and load the results into Pandas DataFrames
store = pd.read_sql(store, connection)
#Show retrieved table
store.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [21]:
# close connection
connection.close()


In [22]:
train_df = pd.read_csv(r'C:\Users\User\Desktop\Azubi_Projects\TIME-SERIES-REGRESSION\Data\train.csv')

train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
